In [2]:
import torch
from transformers import BigBirdForQuestionAnswering, BigBirdTokenizer

In [4]:
model_id = "vasudevgupta/bigbird-base-trivia-itc"
model = BigBirdForQuestionAnswering.from_pretrained(model_id, block_size=16, num_random_blocks=3)
tokenizer = BigBirdTokenizer.from_pretrained(model_id)

Downloading: 100%|██████████| 846k/846k [00:01<00:00, 842kB/s] 
Downloading: 100%|██████████| 943/943 [00:00<00:00, 212kB/s]


In [37]:
context = "🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between TensorFlow 2.0 and PyTorch. Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset"

In [38]:
def get_answer(question, context):
    encoding = tokenizer(question, context, return_tensors="pt", max_length=128, padding="max_length", truncation=True)
    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask

    with torch.no_grad():
        start_scores, end_scores = model(input_ids=input_ids, attention_mask=attention_mask).to_tuple()

    # Let's take the most likely token using `argmax` and retrieve the answer
    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

    answer_tokens = all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1]
    answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

    return answer

In [39]:
question = "How many pretrained models are available in 🤗 Transformers?"
get_answer(question, context)

'32'